In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from layer import Grain, Layer
from snowProfile import (
    SnowProfile,
    SurfaceCondition,
    Layer,
    TempObs,
    DensityObs,
)
from whumpfData import WumphData
from coreInfo import CoreInfo, User, Location, WeatherConditions

## Try other parser

In [18]:
import xml.etree.ElementTree as ET

file_path = "snowpits/test/snowpylot-test-26-Feb-caaml.xml"
# file_path = "snowpits/test/mkc_TESTPIT-23-Feb-caaml.xml"
# file_path = "snowpits/test/snowpits-13720-caaml.xml"

caaml_tag = (
    "{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}"  # Update to read from xml file
)
gml_tag = "{http://www.opengis.net/gml}"
snowpilot_tag = "{http://www.snowpilot.org/Schemas/caaml}"
root = ET.parse(file_path).getroot()

In [37]:
surfCond = next(root.iter(caaml_tag + "surfCond"), None)
print(surfCond)

if surfCond is not None:
    print("not None")

    # windLoading
    for prop in surfCond.iter(snowpilot_tag + "windLoading"):
        print(prop.text)

<Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}surfCond' at 0x00000232D292E1B0>
not None
previous
